## Fetch EAS attestations from HyperSync

### Setup

In [190]:
import hypersync
from hypersync import (
    LogSelection,
    LogField,
    BlockField,
    FieldSelection,
    HexOutput
)
import os, glob
import pandas as pd
import json

In [191]:
def get_envio_key():
    return os.getenv("HYPERSYNC_API_KEY")

In [192]:
# Define a dictionary of key blockchain name and value a dictionary of contract address, eascan url and hypersync url
blockchain_networks = {
    "arbitrum": {
        "contract_address": "0xbD75f629A22Dc1ceD33dDA0b68c546A1c035c458",
        "easscan_url": "https://arbitrum.easscan.org",
        "hypersync_url": "https://arbitrum.hypersync.xyz"
    },
    "base": {
        "contract_address": "0x4200000000000000000000000000000000000021",
        "easscan_url": "https://base.easscan.org",
        "hypersync_url": "https://base.hypersync.xyz"
    },
    "optimism": {
        "contract_address": "0x4200000000000000000000000000000000000021",
        "easscan_url": "https://optimism.easscan.org",
        "hypersync_url": "https://optimism.hypersync.xyz"
    },
    "polygon": {
        "contract_address": "0x5E634ef5355f45A855d02D66eCD687b1502AF790",
        "easscan_url": "https://polygon.easscan.org",
        "hypersync_url": "https://polygon.hypersync.xyz"
    },
    "mainnet": {
        "contract_address": "0xA1207F3BBa224E2c9c3c6D5aF63D0eb1582Ce587",
        "easscan_url": "https://easscan.org",
        "hypersync_url": "https://eth.hypersync.xyz"
    },
    "sepolia": {
        "contract_address": "0xC2679fBD37d54388Ce493F1DB75320D236e1815e",
        "easscan_url": "https://sepolia.easscan.org",
        "hypersync_url": "https://sepolia.hypersync.xyz"
    }
}

In [193]:
def get_contract_address(blockchain_name):
    return blockchain_networks[blockchain_name]["contract_address"]

In [194]:
def get_eas_graph_ql_url(blockchain_name):
    return blockchain_networks[blockchain_name]["easscan_url"] + "/graphql"

In [195]:
def get_hypersync_url(blockchain_name):
    return blockchain_networks[blockchain_name]["hypersync_url"]


### Collect events with Hypersync

In [196]:
from hypersync import JoinMode


def read_attestation_events(root_dir):
    # Read the attestations from the parquet file
    log_parts = glob.glob(f"{root_dir}/**/logs.parquet", recursive=True)
    df = pd.concat([pd.read_parquet(p) for p in log_parts], ignore_index=True)

    print(f"Found {len(df)} total attestations")

    return df

async def collect_attestation_events(start_block, end_block,blockchain_name="arbitrum"):
    hypersync_api_key = get_envio_key()
    # The event signature string
    event_signature = "Attested(address indexed recipient, address indexed attester, bytes32 uid, bytes32 indexed schema)"

    # Initialize client
    client = hypersync.HypersyncClient(
        hypersync.ClientConfig(
            url=get_hypersync_url(blockchain_name),  # Arbitrum network for EAS
            bearer_token=hypersync_api_key,
        )
    )

    # Define field selection
    field_selection = FieldSelection(
        log=[
            LogField.ADDRESS,
            LogField.TOPIC0,
            LogField.TOPIC1,
            LogField.TOPIC2,
            LogField.TOPIC3,
            LogField.DATA,
            LogField.BLOCK_NUMBER
        ]
    )

    # Define query for EAS Attested events
    query = hypersync.Query(
        from_block=0,  # Start from genesis
        to_block=100000000,  # Large number to get recent blocks
        field_selection=field_selection,
        join_mode= JoinMode.JOIN_ALL,
        logs=[
            LogSelection(
                address=[get_contract_address(blockchain_name)],  # EAS contract on Arbitrum
                topics=[
                    ["0x8bf46bf4cfd674fa735a3d63ec1c9ad4153f033c290341f3a588b75685141b35"]  # Attested event signature
                ]
            )
        ]
    )

    # Configure output
    config = hypersync.StreamConfig(
        hex_output=HexOutput.PREFIXED,
        event_signature=event_signature
    )

    root_dir = f"./eas_attestations/{blockchain_name}/logs_dataset"
    os.makedirs(root_dir, exist_ok=True)

    # Collect data to a Parquet file
    current_block = start_block
    while current_block < end_block:
        query.from_block = current_block
        query.to_block = min(current_block + 1_000_000, end_block)
        out_dir = f"{root_dir}/chunk_{query.from_block}_{query.to_block}"
        print(f"Processing blocks {query.from_block}→{query.to_block} out of {end_block} → {out_dir}")
        await client.collect_parquet(out_dir, query, config)  # writes logs.parquet, blocks.parquet, etc. into out_dir
        current_block = query.to_block + 1
        
    print(f"Processed blocks {query.from_block} to {query.to_block}")
    df = read_attestation_events(root_dir)

    return len(df)

    

In [197]:
# Run the function
await collect_attestation_events(386757242-10000000,386757242+1)

Processing blocks 376757242→377757242 out of 386757243 → ./eas_attestations/arbitrum/logs_dataset/chunk_376757242_377757242
Processing blocks 377757243→378757243 out of 386757243 → ./eas_attestations/arbitrum/logs_dataset/chunk_377757243_378757243
Processing blocks 378757244→379757244 out of 386757243 → ./eas_attestations/arbitrum/logs_dataset/chunk_378757244_379757244
Processing blocks 379757245→380757245 out of 386757243 → ./eas_attestations/arbitrum/logs_dataset/chunk_379757245_380757245
Processing blocks 380757246→381757246 out of 386757243 → ./eas_attestations/arbitrum/logs_dataset/chunk_380757246_381757246
Processing blocks 381757247→382757247 out of 386757243 → ./eas_attestations/arbitrum/logs_dataset/chunk_381757247_382757247
Processing blocks 382757248→383757248 out of 386757243 → ./eas_attestations/arbitrum/logs_dataset/chunk_382757248_383757248
Processing blocks 383757249→384757249 out of 386757243 → ./eas_attestations/arbitrum/logs_dataset/chunk_383757249_384757249
Processi

1489

### Process events

In [198]:
def last_40_addr(topic):
    if not isinstance(topic, str):
        return None
    h = topic[2:] if topic.startswith('0x') else topic
    if len(h) < 40:
        return None
    return '0x' + h[-40:]
    
def process_attestation_events(blockchain_name="arbitrum"):
    root_dir = f"./eas_attestations/{blockchain_name}/logs_dataset"

    # Read the attestations from the parquet file
    df = read_attestation_events(root_dir)
    
    # Decode the attestation data
    decoded_attestations = []
    
    for index, row in df.iterrows():
        # Decode the topics to get the indexed parameters
        recipient = last_40_addr(row['topic1'])
        attester = last_40_addr(row['topic2'])
        schema = row['topic3']                  # Full topic3 is the schema
        
        # Decode the data field to get the uid (first 32 bytes)
        data_hex = row['data'][2:]  # Remove 0x prefix
        uid = "0x" + data_hex[:64]  # First 32 bytes (64 hex chars)
        
        attestation = {
            "recipient": recipient,
            "attester": attester,
            "uid": uid,
            "schema": schema,
            "block_number": row.get('block_number', 'N/A'),
            "raw_data": row['data'],
            "raw_topics": {
                "topic0": row['topic0'],
                "topic1": row['topic1'],
                "topic2": row['topic2'],
                "topic3": row['topic3']
            }
        }
        
        decoded_attestations.append(attestation)
    
    # Save as JSON
    with open(f'./eas_attestations/{blockchain_name}/processed_attestation_events.json', 'w') as f:
        json.dump(decoded_attestations, f, indent=2)
    
    print(f"Processed {len(decoded_attestations)} attestations and saved to eas_attestations/{blockchain_name}/processed_attestation_events.json")
    
    # Print first few for inspection
    print("\nFirst few processed attestations:")
    for i, att in enumerate(decoded_attestations[:3]):
        print(f"\nAttestation {i+1}:")
        print(f"  Recipient: {att['recipient']}")
        print(f"  Attester: {att['attester']}")
        print(f"  UID: {att['uid']}")
        print(f"  Schema: {att['schema']}")
    
    return decoded_attestations


In [199]:
process_attestation_events("arbitrum")

Found 1489 total attestations
Processed 1489 attestations and saved to eas_attestations/arbitrum/processed_attestation_events.json

First few processed attestations:

Attestation 1:
  Recipient: 0x3fabf17555376568b20c8688b708b691553f339d
  Attester: 0x7848a3578ff2e1f134659a23f64a404a4d710475
  UID: 0x9fa9c36cd6952fe2ef0663a2023f0d97525c0bacd14fca06d568b8a190007976
  Schema: 0x1f3dce6501d8aad23563c0cf4f0c32264aed9311cb050056ebf72774f89ba912

Attestation 2:
  Recipient: None
  Attester: None
  UID: 0x
  Schema: None

Attestation 3:
  Recipient: 0x72009d5cedfb1173cf459d03eabea86cbf0e8e6e
  Attester: 0x7848a3578ff2e1f134659a23f64a404a4d710475
  UID: 0x21522385b351df337365596ad7126be8f087594f653ed79e669925a1bbe3cdda
  Schema: 0x1f3dce6501d8aad23563c0cf4f0c32264aed9311cb050056ebf72774f89ba912


[{'recipient': '0x3fabf17555376568b20c8688b708b691553f339d',
  'attester': '0x7848a3578ff2e1f134659a23f64a404a4d710475',
  'uid': '0x9fa9c36cd6952fe2ef0663a2023f0d97525c0bacd14fca06d568b8a190007976',
  'schema': '0x1f3dce6501d8aad23563c0cf4f0c32264aed9311cb050056ebf72774f89ba912',
  'block_number': 380759234,
  'raw_data': '0x9fa9c36cd6952fe2ef0663a2023f0d97525c0bacd14fca06d568b8a190007976',
  'raw_topics': {'topic0': '0x8bf46bf4cfd674fa735a3d63ec1c9ad4153f033c290341f3a588b75685141b35',
   'topic1': '0x0000000000000000000000003fabf17555376568b20c8688b708b691553f339d',
   'topic2': '0x0000000000000000000000007848a3578ff2e1f134659a23f64a404a4d710475',
   'topic3': '0x1f3dce6501d8aad23563c0cf4f0c32264aed9311cb050056ebf72774f89ba912'}},
 {'recipient': None,
  'attester': None,
  'uid': '0x',
  'schema': None,
  'block_number': 380768565,
  'raw_data': '0x',
  'raw_topics': {'topic0': '0xbb47ee3e183a558b1a2ff0874b079f3fc5478b7454eacf2bfc5af2ff5878f972',
   'topic1': None,
   'topic2': None,

## Enrich Attestation Data with data from EAS GraphQL

In [200]:
import requests

### fetch_attestation_eas_graph_ql_by_uid (unused)

In [201]:
async def fetch_attestation_eas_graph_ql_by_uid(uid,blockchain_name="arbitrum"):
    """
    Fetch attestation data from EAS API by UID
    
    Args:
        uid (str): The attestation UID to fetch
        
    Returns:
        dict: The attestation data or None if not found
    """
    
    # EAS GraphQL API endpoint
    url = get_eas_graph_ql_url(blockchain_name)
    
    # GraphQL query to fetch attestation by UID (correct structure)
    query = """
        query Attestation($id: String!) {
            attestation(where: { id: $id }) {
                id
                attester
                recipient
                refUID
                revocable
                revocationTime
                expirationTime
                data
            }
        }
    """
    
    # Variables for the query
    variables = {
        "id": uid
    }
    
    # Request payload
    payload = {
        "query": query,
        "variables": variables
    }
    
    try:
        # Make the request
        response = requests.post(
            url,
            json=payload,
            headers={"Content-Type": "application/json"}
        )
        
        # Check if request was successful
        response.raise_for_status()
        
        # Parse the response
        data = response.json()
        
        # Check for GraphQL errors
        if "errors" in data:
            print(f"GraphQL errors: {data['errors']}")
            return None
            
        # Return the attestation data
        attestation = data.get("data", {}).get("attestation")
        
        if attestation is None:
            print(f"No attestation found with UID: {uid}")
            return None
            
        print(f"Successfully fetched attestation for UID: {uid}")
        return attestation
        
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON response: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

In [202]:
# Test function with one of the UIDs from your data
test_uid = "0x3ffcecf3fd79254919ddba80370729327f44472a296e0e8f54e3996652b38d51"  # Replace with actual UID
result = await fetch_attestation_eas_graph_ql_by_uid(test_uid)

if result:
    print("Attestation data:")
    print(json.dumps(result, indent=2))
else:
    print("Failed to fetch attestation data")

Successfully fetched attestation for UID: 0x3ffcecf3fd79254919ddba80370729327f44472a296e0e8f54e3996652b38d51
Attestation data:
{
  "id": "0x3ffcecf3fd79254919ddba80370729327f44472a296e0e8f54e3996652b38d51",
  "attester": "0xB2331EAad45730CD8CC3A553E3eC43257829DC03",
  "recipient": "0xB2331EAad45730CD8CC3A553E3eC43257829DC03",
  "refUID": "0x5632ba1bd850239423138587d3862ff89f67c982d2e311132954a877541b3383",
  "revocable": true,
  "revocationTime": 0,
  "expirationTime": 0,
  "data": "0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000005d7b2274797065223a22636f6d706c65746564222c22726561736f6e223a22222c2270726f6f664f66576f726b223a22222c22636f6d706c6574696f6e50657263656e74616765223a22222c2264656c6976657261626c6573223a5b5d7d000000"
}


### fetch_attestation_eas_graph_ql_for_multiple_uids

In [ ]:
# This function should take a list of attestations and return a list of enriched attestations with schema, block_number and blockchain_name
async def fetch_attestation_eas_graph_ql_for_multiple_uids(attestation_events,blockchain_name="arbitrum"):
    """
    Fetch attestation data from EAS API by UID
    
    Args:
        uid (str): The attestation UID to fetch
        
    Returns:
        dict: The attestation data or None if not found
    """
    
    # EAS GraphQL API endpoint
    url = get_eas_graph_ql_url(blockchain_name)
    
    # GraphQL query to fetch attestation by UID (correct structure)
    query = """
        query Attestations($ids: [String!]!) {
            attestations(where: { id:{in: $ids }}) {
                id
                attester
                recipient
                refUID
                revocable
                revocationTime
                expirationTime
                data
            }
        }
    """
    
    # Variables for the query
    variables = {
        "ids": attestation_events.uid.tolist()
    }
    
    # Request payload
    payload = {
        "query": query,
        "variables": variables
    }
    
    try:
        # Make the request
        response = requests.post(
            url,
            json=payload,
            headers={"Content-Type": "application/json"}
        )
        
        # Check if request was successful
        response.raise_for_status()
        
        # Parse the response
        data = response.json()
        
        # Check for GraphQL errors
        if "errors" in data:
            print(f"GraphQL errors: {data['errors']}")
            return None
            
        # Return the attestation data
        attestations = data.get("data", {}).get("attestations")
        
        if attestations is None:
            print(f"No attestation found with UIDs: {attestation_events.uid.tolist()}")
            return None
            
        print(f"Successfully fetched attestation for UIDs: {attestation_events.uid.tolist()}")
        # Enrich attestations with additional metadata
        enriched_attestations = []
        
        # dedupe latest per uid
        aed = (
            attestation_events.sort_values('block_number')
            .groupby('uid', as_index=False)
            .last()[['uid', 'schema', 'block_number']]
        )

        lookup = aed.set_index('uid')[['schema', 'block_number']].to_dict('index')

        enriched_attestations = []
        for att in attestations:
            meta = lookup.get(att['id'])
            if not meta:
                # skip or set None defaults
                # enriched_attestations.append({**att, 'schema': None, 'block_number': None})
                continue
            enriched_attestations.append({
                **att,
                'schema': meta['schema'],
                'block_number': meta['block_number'],
                'blockchain_name': blockchain_name,
            })
        return enriched_attestations
        
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON response: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

async def fetch_attestation_eas_graph_ql_for_multiple_uids_in_chunks(uids,blockchain_name="arbitrum",chunk_size=100):
    """
    Fetch attestation data from EAS API in chunks of a specified size
    
    Args:
        uids (list): List of attestation UIDs to fetch
        chunk_size (int): Number of UIDs per chunk
        
    Returns:
        list: List of attestation data or None if no data is found
    """

    all_attestations = []
    nb_of_fetched_attestations = 0
    for i in range(0, len(uids), chunk_size):
        chunk_uids = uids[i:i+chunk_size]
        print(f"Fetching attestations {nb_of_fetched_attestations} to {nb_of_fetched_attestations+len(chunk_uids)} out of {len(uids)} for {blockchain_name}")
        nb_of_fetched_attestations += len(chunk_uids)
        attestations = await fetch_attestation_eas_graph_ql_for_multiple_uids(chunk_uids,blockchain_name)
        if attestations:
            all_attestations.extend(attestations)
    
    return all_attestations


In [223]:
# Test function with one of the UIDs from your data
test_uids = ["0x7530011d80d3b2e66efe436ee57b4ecd33bd54f36480d992d5e5ac63fea85c1a", "0x9fa9c36cd6952fe2ef0663a2023f0d97525c0bacd14fca06d568b8a190007976"]
blockchain_name = "arbitrum"

# Fetch the attestation events for the test UIDs
df_test = pd.read_json(f"./eas_attestations/{blockchain_name}/processed_attestation_events.json")
df_test = df_test[df_test['uid'].isin(test_uids)]
df_test= df_test.groupby('uid', as_index=False).last()
print("\Input contents:")
display(df_test.style.set_properties(**{'text-align': 'left'})
       .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}]))

result = await fetch_attestation_eas_graph_ql_for_multiple_uids_in_chunks(df_test,blockchain_name)

if result:
    print("Attestation data:")
    print(json.dumps(result, indent=2, default=str))
else:
    print("Failed to fetch attestation data")


\Input contents:


<>:9: SyntaxWarning: invalid escape sequence '\I'
/var/folders/hd/mwwk_4zs7blcj6tc0ycm9t5c0000gn/T/ipykernel_21407/845200827.py:9: SyntaxWarning: invalid escape sequence '\I'
  print("\Input contents:")


,uid,recipient,attester,schema,block_number,raw_data,raw_topics
0,0x7530011d80d3b2e66efe436ee57b4ecd33bd54f36480d992d5e5ac63fea85c1a,0xf5ca53792c47e3a0792380292d15c894097015ff,0x6dc1d6b864e8bef815806f9e4677123496e12026,0x16bfe4783b7a9c743c401222c56a07ecb77ed42afc84b61ff1f62f5936c0b9d7,383225167,0x7530011d80d3b2e66efe436ee57b4ecd33bd54f36480d992d5e5ac63fea85c1a,"{'topic0': '0xdb70a95bf9cd8aba06a6279fcf4478bc043686a8d579b0dc09f623b7b1289729', 'topic1': '0x000000000000000000000000f5ca53792c47e3a0792380292d15c894097015ff', 'topic2': None, 'topic3': None}"
1,0x9fa9c36cd6952fe2ef0663a2023f0d97525c0bacd14fca06d568b8a190007976,0x3fabf17555376568b20c8688b708b691553f339d,0x7848a3578ff2e1f134659a23f64a404a4d710475,0x1f3dce6501d8aad23563c0cf4f0c32264aed9311cb050056ebf72774f89ba912,380759234,0x9fa9c36cd6952fe2ef0663a2023f0d97525c0bacd14fca06d568b8a190007976,"{'topic0': '0x8bf46bf4cfd674fa735a3d63ec1c9ad4153f033c290341f3a588b75685141b35', 'topic1': '0x0000000000000000000000003fabf17555376568b20c8688b708b691553f339d', 'topic2': '0x0000000000000000000000007848a3578ff2e1f134659a23f64a404a4d710475', 'topic3': '0x1f3dce6501d8aad23563c0cf4f0c32264aed9311cb050056ebf72774f89ba912'}"


Fetching attestations 0 to 2 out of 2 for arbitrum
Successfully fetched attestation for UIDs: ['0x7530011d80d3b2e66efe436ee57b4ecd33bd54f36480d992d5e5ac63fea85c1a', '0x9fa9c36cd6952fe2ef0663a2023f0d97525c0bacd14fca06d568b8a190007976']
Attestation data:
[
  {
    "id": "0x7530011d80d3b2e66efe436ee57b4ecd33bd54f36480d992d5e5ac63fea85c1a",
    "attester": "0x6dC1D6b864e8BEf815806f9e4677123496e12026",
    "recipient": "0xF5CA53792C47e3a0792380292D15c894097015fF",
    "refUID": "0xca93275f385474a2c49612f46b09ac2ff3014e565a29f38b681922068d1674e8",
    "revocable": true,
    "revocationTime": 0,
    "expirationTime": 0,
    "data": "0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000182c7b227469746c65223a22426561636f6e204c616273222c226465736372697074696f6e223a22426561636f6e204c6162732028707265762046726163746f6e2052657365617263682920697320616e2052264420696e737469747574696f6e20666f72206120706f7369746976652073756d20776f72

### decode_attestation_data

In [205]:
import json

def decode_attestation_data(hex_data):
    """
    Decode the data field from an EAS attestation
    
    Args:
        hex_data (str): The hex-encoded data field from the attestation
        
    Returns:
        dict: Decoded data or None if decoding fails
    """
    try:
        # Remove 0x prefix if present
        if hex_data.startswith('0x'):
            hex_data = hex_data[2:]
        
        # Convert hex to bytes
        data_bytes = bytes.fromhex(hex_data)
        
        # The data is typically ABI-encoded
        # First, let's try to decode it as a string (common for EAS attestations)
        
        # Skip the first 32 bytes (offset) and next 32 bytes (length)
        # Then read the actual string data
        if len(data_bytes) > 64:  # At least 64 bytes (32 + 32)
            # Get the length of the string (second 32 bytes)
            length_hex = data_bytes[32:64]
            length = int.from_bytes(bytes.fromhex(length_hex.hex()), 'big')
            
            # Extract the string data
            if length > 0 and len(data_bytes) >= 64 + length:
                string_data = data_bytes[64:64+length]
                
                try:
                    # Try to decode as UTF-8 string
                    decoded_string = string_data.decode('utf-8')
                    
                    # Try to parse as JSON if it looks like JSON
                    if decoded_string.strip().startswith('{'):
                        return json.loads(decoded_string)
                    else:
                        return {"decoded_string": decoded_string}
                        
                except (UnicodeDecodeError, json.JSONDecodeError):
                    # If it's not a string or JSON, return the raw hex
                    return {"raw_hex": hex_data, "raw_bytes": data_bytes.hex()}
        
        # If we can't decode as string, return raw data
        return {"raw_hex": hex_data, "raw_bytes": data_bytes.hex()}
        
    except Exception as e:
        print(f"Error decoding data: {e}")
        return {"error": str(e), "raw_hex": hex_data}

# Test with your attestation data
test_data = "0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000005d7b2274797065223a22636f6d706c65746564222c22726561736f6e223a22222c2270726f6f664f66576f726b223a22222c22636f6d706c6574696f6e50657263656e74616765223a22222c2264656c6976657261626c6573223a5b5d7d000000"

decoded = decode_attestation_data(test_data)
print("Decoded attestation data:")
print(json.dumps(decoded, indent=2))


Decoded attestation data:
{
  "type": "completed",
  "reason": "",
  "proofOfWork": "",
  "completionPercentage": "",
  "deliverables": []
}


### enrich_attestation_data

In [224]:
import json
import numpy as np
import pandas as pd
from datetime import datetime
from decimal import Decimal

def to_jsonable(o):
    if isinstance(o, (np.integer,)):
        return int(o)
    if isinstance(o, (np.floating,)):
        return float(o)
    if isinstance(o, (np.bool_,)):
        return bool(o)
    if isinstance(o, (pd.Timestamp, datetime)):
        return o.isoformat()
    if isinstance(o, Decimal):
        return float(o)
    if isinstance(o, (np.ndarray,)):
        return o.tolist()
    return str(o)

async def enrich_attestation_data(blockchain_name="arbitrum"):
    # Read the processed attestation events
    df = pd.read_json(f"./eas_attestations/{blockchain_name}/processed_attestation_events.json").groupby('uid', as_index=False).last()
    print(len(df))
    enriched_attestations = await fetch_attestation_eas_graph_ql_for_multiple_uids_in_chunks(df,blockchain_name,10000)
    print(len(enriched_attestations))
    # Decode the attestation data
    enriched_attestations_decoded = []
    if enriched_attestations is not None:
        for attestation in enriched_attestations:
            if attestation is not None:
                attestation['decoded_data'] = decode_attestation_data(attestation['data'])
                # Check if the decoded data, converted to string, contains "\u0000"
                if "\u0000" not in str(attestation['decoded_data']):
                    enriched_attestations_decoded.append(attestation)

    # log the number of lines in the file before appending, or none if the file does not exist
    if os.path.exists(f"./eas_attestations/enriched_attestation_events.jsonl"):
        nb_of_lines = sum(1 for line in open(f"./eas_attestations/enriched_attestation_events.jsonl"))
        print(f"Number of lines in the file before appending: {nb_of_lines}")
    else:
        print("File does not exist yet")
        
    # save the enriched_attestations_decoded to a json file
    with open(f"./eas_attestations/enriched_attestation_events.jsonl", 'a') as f:
        for attestation_decoded in enriched_attestations_decoded:
            f.write(json.dumps(attestation_decoded, default=to_jsonable) + "\n")

    # log the number of lines in the file after appending, or none if the file does not exist
    print(f"Processed {len(enriched_attestations_decoded)} for {blockchain_name} attestations and saved to eas_attestations/enriched_attestation_events.json")

    nb_of_lines = sum(1 for line in open(f"./eas_attestations/enriched_attestation_events.jsonl"))
    print(f"Number of lines in the file after appending: {nb_of_lines}")
    
    
    return enriched_attestations_decoded
    

In [225]:
await enrich_attestation_data()

1314
Fetching attestations 0 to 1314 out of 1314 for arbitrum
Successfully fetched attestation for UIDs: ['0x', '...']
1286
Number of lines in the file before appending: 3858
Processed 1286 for arbitrum attestations and saved to eas_attestations/enriched_attestation_events.json
Number of lines in the file after appending: 5144


[{'id': '0x7530011d80d3b2e66efe436ee57b4ecd33bd54f36480d992d5e5ac63fea85c1a',
  'attester': '0x6dC1D6b864e8BEf815806f9e4677123496e12026',
  'recipient': '0xF5CA53792C47e3a0792380292D15c894097015fF',
  'refUID': '0xca93275f385474a2c49612f46b09ac2ff3014e565a29f38b681922068d1674e8',
  'revocable': True,
  'revocationTime': 0,
  'expirationTime': 0,
  'data': '0x0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000182c7b227469746c65223a22426561636f6e204c616273222c226465736372697074696f6e223a22426561636f6e204c6162732028707265762046726163746f6e2052657365617263682920697320616e2052264420696e737469747574696f6e20666f72206120706f7369746976652073756d20776f726c64207468726f756768206578706c6f72696e6720696e636c757369766520636f6f7264696e6174696f6e2064657369676e2e2057652077696c6c206272696e6720706c7572616c2066756e64696e67206d656368616e69736d73207265666c656374656420706c7572616c2076616c75657320696e746f2061207075626c696320676f6f647320737

## Filter attestations containing "github"

In [208]:
def filter_attestations_containing_github():
    # Read the enriched attestation events
    df = pd.read_json("./eas_attestations/enriched_attestation_events.jsonl",lines=True)

    # Filter attestations containing "github" (decoded data is a json and must be converted to string first)
    df_filtered = df[df['decoded_data'].apply(lambda x: str(x)).str.contains('github')]

    print(f"Found {len(df_filtered)} attestations containing 'github'")
    
    # Save the filtered data
    df_filtered.to_json("./eas_attestations/filtered_attestation_events.jsonl", orient='records', lines=True)
    
    return df

In [209]:
# filter_attestations_containing_github()

## Full Data process

### full_data_process

In [210]:
async def full_data_process(start_block, end_block, blockchain_name="arbitrum"):
    # Collect EAS attestations
    await collect_attestation_events(start_block, end_block,blockchain_name)

    # Process EAS attestations
    process_attestation_events(blockchain_name)

    # Enrich EAS attestations
    await enrich_attestation_data(blockchain_name)

    # Filter attestations containing "github"
    filter_attestations_containing_github()
    
    return

### Run

#### mainnet

In [211]:
# mainnet
# lastblock=23570552
# await full_data_process(0,lastblock,"mainnet")

#### arbitrum

In [212]:
# arbitrum
# lastblock=389183720
# await full_data_process(0,lastblock,"arbitrum")

#### optimism

In [213]:
# optimism
# lastblock=142391425
# await full_data_process(0,lastblock,"optimism")

#### base

In [214]:
# base
# lastblock=36820269
# await full_data_process(0,lastblock,"base")